# RNN GRU Explainer

The goal is to walk through RNNs (recurring neural networks), there are 2 flavors: GRUs and LSTMs. We'll focus on Gated Recurrent Unit (GRUs).

To help display the transformation, we'll use the first sentence from the [linear algebra wiki page](https://en.wikipedia.org/wiki/Linear_algebra) and [lu decomposition wiki page](https://en.wikipedia.org/wiki/LU_decomposition) as the topic is fitting and it shows us some non-standard patterns.  

## Text Prep/Tokenization

we'll start with common preprocessing step of tokenizing the data.  This converts the string text into an array of numbers that can be used during the training loop.  I've built a very subtle byte-pair encdoing that has each unique character that appears and the top 5 merges. This keeps our vocab size small and managable for this example. Typically the vocab size is in the 100K+ range. A great library for this is `tiktoken`. Tokenization simply finds the longest pattern of characters that's in common with what was trained and replaces it with an integer that represents it.  This way we turn the text into a numeric array to simplify computing. import torch
from collections import Counter

In [1]:
import torch
from collections import Counter

In [2]:
class SimpleBPETokenizer:
    def __init__(self, num_merges=5, eot_token='<|endoftext|>'):
        self.num_merges = num_merges
        self.eot_token = eot_token
        self.eot_id = None
        self.merges = []
        self.pair_ranks = {}
        self.vocab = {}
        self.id_to_token = {}

    def _add_token(self, tok):
        if tok in self.vocab:
            return self.vocab[tok]
        i = len(self.vocab)
        self.vocab[tok] = i
        self.id_to_token[i] = tok
        return i

    def _get_bigrams(self, seq):
        for i in range(len(seq) - 1):
            yield (seq[i], seq[i + 1])

    def _merge_once(self, seq, pair):
        a, b = pair
        out = []
        i = 0
        while i < len(seq):
            if i < len(seq) - 1 and seq[i] == a and seq[i + 1] == b:
                out.append(a + b)
                i += 2
            else:
                out.append(seq[i])
                i += 1
        return out

    def train(self, corpus):
        # corpus: list[str]
        text = ''.join(corpus).lower()
        seq = list(text)
        merges = []
        for _ in range(self.num_merges):
            counts = Counter(self._get_bigrams(seq))
            if not counts: break
            best_pair, _ = counts.most_common(1)[0]
            merges.append(best_pair)
            seq = self._merge_once(seq, best_pair)
        self.merges = merges
        self.pair_ranks = {p: i for i, p in enumerate(self.merges)}

        self.vocab = {}
        self.id_to_token = {}
        for ch in sorted(set(text)):
            self._add_token(ch)
        for a, b in self.merges:
            self._add_token(a + b)
        self.eot_id = self._add_token(self.eot_token)

    def encode(self, text, force_last_eot=True):
        # treat literal eot marker as special; remove it from content
        if self.eot_token in text:
            text = text.replace(self.eot_token, '')
        seq = list(text)

        # make sure all seen base chars exist
        for ch in set(seq):
            if ch not in self.vocab:
                self._add_token(ch)

        # greedy BPE using learned pair ranks
        if self.merges:
            while True:
                best_pair, best_rank = None, None
                for p in self._get_bigrams(seq):
                    r = self.pair_ranks.get(p)
                    if r is not None and (best_rank is None or r < best_rank):
                        best_pair, best_rank = p, r
                if best_pair is None:
                    break
                seq = self._merge_once(seq, best_pair)

        # ensure all tokens in seq exist in vocab (e.g., if new chars appeared)
        for tok in seq:
            if tok not in self.vocab:
                self._add_token(tok)

        ids = [self.vocab[tok] for tok in seq]

        # FORCE: append EOT id if not already last
        if force_last_eot:
            if not ids or ids[-1] != self.eot_id:
                ids.append(self.eot_id)

        return ids

    def decode(self, ids):
        # drop trailing EOT if present
        if ids and self.eot_id is not None and ids[-1] == self.eot_id:
            ids = ids[:-1]
        toks = [self.id_to_token[i] for i in ids]
        return ''.join(toks)


In [3]:
raw_example_1 = r'''Linear algebra is central to almost all areas of mathematics. For instance, linear algebra is fundamental in modern presentations of geometry, including for defining basic objects such as lines, planes and rotations. Also, functional analysis, a branch of mathematical analysis, may be viewed as the application of linear algebra to function spaces.'''
raw_example_2 = r'''In numerical analysis and linear algebra, lower–upper (LU) decomposition or factorization factors a matrix as the product of a lower triangular matrix and an upper triangular matrix (see matrix multiplication and matrix decomposition).'''


In [4]:
tok = SimpleBPETokenizer(num_merges=5)
tok.train([raw_example_1,raw_example_2])
tok.merges

[(' ', 'a'), ('a', 't'), ('i', 'n'), (' ', 'm'), ('i', 'o')]

In [5]:
tok.vocab

{' ': 0,
 '(': 1,
 ')': 2,
 ',': 3,
 '.': 4,
 'a': 5,
 'b': 6,
 'c': 7,
 'd': 8,
 'e': 9,
 'f': 10,
 'g': 11,
 'h': 12,
 'i': 13,
 'j': 14,
 'l': 15,
 'm': 16,
 'n': 17,
 'o': 18,
 'p': 19,
 'r': 20,
 's': 21,
 't': 22,
 'u': 23,
 'v': 24,
 'w': 25,
 'x': 26,
 'y': 27,
 'z': 28,
 '–': 29,
 ' a': 30,
 'at': 31,
 'in': 32,
 ' m': 33,
 'io': 34,
 '<|endoftext|>': 35}

In [6]:
vocab_size = len(tok.vocab)
vocab_size

36

In [7]:
eot = tok.eot_id
tokens = []
for example in [raw_example_1, raw_example_2]:
    tokens.extend([eot])
    tokens.extend(tok.encode(example.lower()))
all_tokens = torch.tensor(tokens, dtype=torch.long)
all_tokens

tensor([35, 15, 32,  9,  5, 20, 30, 15, 11,  9,  6, 20,  5,  0, 13, 21,  0,  7,
         9, 17, 22, 20,  5, 15,  0, 22, 18, 30, 15, 16, 18, 21, 22, 30, 15, 15,
        30, 20,  9,  5, 21,  0, 18, 10, 33, 31, 12,  9, 16, 31, 13,  7, 21,  4,
         0, 10, 18, 20,  0, 32, 21, 22,  5, 17,  7,  9,  3,  0, 15, 32,  9,  5,
        20, 30, 15, 11,  9,  6, 20,  5,  0, 13, 21,  0, 10, 23, 17,  8,  5, 16,
         9, 17, 22,  5, 15,  0, 32, 33, 18,  8,  9, 20, 17,  0, 19, 20,  9, 21,
         9, 17, 22, 31, 34, 17, 21,  0, 18, 10,  0, 11,  9, 18, 16,  9, 22, 20,
        27,  3,  0, 32,  7, 15, 23,  8, 32, 11,  0, 10, 18, 20,  0,  8,  9, 10,
        32, 32, 11,  0,  6,  5, 21, 13,  7,  0, 18,  6, 14,  9,  7, 22, 21,  0,
        21, 23,  7, 12, 30, 21,  0, 15, 32,  9, 21,  3,  0, 19, 15,  5, 17,  9,
        21, 30, 17,  8,  0, 20, 18, 22, 31, 34, 17, 21,  4, 30, 15, 21, 18,  3,
         0, 10, 23, 17,  7, 22, 34, 17,  5, 15, 30, 17,  5, 15, 27, 21, 13, 21,
         3, 30,  0,  6, 20,  5, 17,  7, 

# Modeling

A machine learning model forward pass now uses the tokenization information, runs several layers of linear algebra on it, and then "predicts" the next token. When it is noisy (like you will see in this example), this process results in gibberish.  The training process changes the noise to pattern during the "backward pass" as you'll see.    We'll show 3 steps that are focused on training:
1. **Data Loading** `x, y = train_loader.next_batch()` - this step pulls from the raw data enough tokens to complete a forward and backward pass.  If the model is inference only, this step is replaced with taking in the inference input and preparing it similarly as the forward pass.
2. **Forward Pass** `logits, loss = model(x, y)` - using the data and the model architecture to predict the next token. When training we also compare against the expected to get loss, but in infrerence, we use the logits to complete the inference task.
3. **Backward Pass & Training** `loss.backward(); optimizer.step()` - using differentials to understand what parameters most impact the forward pass' impact on its prediction, comparing that against what is actually right based on the data loading step, and then making very minor adjustments to the impactful parameters with the hope it improves future predictions.

The we'll show a final **Forward Pass** with the updated weights we did in #3. 

## Data Loading

To start, we need to get enough data to run the forward and backward passes.  Since our total dataset is likely too big to hold all at once in real practice, we would read just enough file information into memory so that we can run the passes, leaving memory and compute to be used on the passes instead of static data holding. 
To start, we have to identify the batch size and the model context length to determine how much data we need.  Consequently, these dimensions also form 2 of the 3 dimensions in the initial matrix.
- **Batch Size (B)** - This is the number of examples you'll train on in a single pass. 
- **Context Length (T)** - This is the max number of tokens that a model can use in a single pass to generat the next token. If an example is below this length, it can be padded.
  
*Ideally both B and T are multiples of 2 to work nicely with chip architecture. This is a common theme across the board*

In [8]:
B = 2 # Batch
T = 8 # context length

In [9]:
current_position = 0
tok_for_training = all_tokens[current_position:current_position + B*T +1 ]
tok_for_training

tensor([35, 15, 32,  9,  5, 20, 30, 15, 11,  9,  6, 20,  5,  0, 13, 21,  0])

In [10]:
tok_for_training

tensor([35, 15, 32,  9,  5, 20, 30, 15, 11,  9,  6, 20,  5,  0, 13, 21,  0])

In [11]:
x=tok_for_training[:-1].view(B, T)
x

tensor([[35, 15, 32,  9,  5, 20, 30, 15],
        [11,  9,  6, 20,  5,  0, 13, 21]])

In [12]:
y=tok_for_training[1:].view(B, T)
y

tensor([[15, 32,  9,  5, 20, 30, 15, 11],
        [ 9,  6, 20,  5,  0, 13, 21,  0]])

## Forward pass

In [13]:
import torch.nn as nn

In [14]:
B_batch, T_context = x.size()
B_batch, T_context

(2, 8)

In [15]:
n_embd = 4 # level of embedding of input tokens
n_embd, vocab_size

(4, 36)

**Embedding input**

Same as with transformer

In [16]:
wte = nn.Embedding(vocab_size, n_embd)
torch.nn.init.ones_(wte.weight)
wte.weight

Parameter containing:
tensor([[1., 1., 1., 1.],
        [1., 1., 1., 1.],
        [1., 1., 1., 1.],
        [1., 1., 1., 1.],
        [1., 1., 1., 1.],
        [1., 1., 1., 1.],
        [1., 1., 1., 1.],
        [1., 1., 1., 1.],
        [1., 1., 1., 1.],
        [1., 1., 1., 1.],
        [1., 1., 1., 1.],
        [1., 1., 1., 1.],
        [1., 1., 1., 1.],
        [1., 1., 1., 1.],
        [1., 1., 1., 1.],
        [1., 1., 1., 1.],
        [1., 1., 1., 1.],
        [1., 1., 1., 1.],
        [1., 1., 1., 1.],
        [1., 1., 1., 1.],
        [1., 1., 1., 1.],
        [1., 1., 1., 1.],
        [1., 1., 1., 1.],
        [1., 1., 1., 1.],
        [1., 1., 1., 1.],
        [1., 1., 1., 1.],
        [1., 1., 1., 1.],
        [1., 1., 1., 1.],
        [1., 1., 1., 1.],
        [1., 1., 1., 1.],
        [1., 1., 1., 1.],
        [1., 1., 1., 1.],
        [1., 1., 1., 1.],
        [1., 1., 1., 1.],
        [1., 1., 1., 1.],
        [1., 1., 1., 1.]], requires_grad=True)

In [17]:
x = wte(x)
x.shape, x

(torch.Size([2, 8, 4]),
 tensor([[[1., 1., 1., 1.],
          [1., 1., 1., 1.],
          [1., 1., 1., 1.],
          [1., 1., 1., 1.],
          [1., 1., 1., 1.],
          [1., 1., 1., 1.],
          [1., 1., 1., 1.],
          [1., 1., 1., 1.]],
 
         [[1., 1., 1., 1.],
          [1., 1., 1., 1.],
          [1., 1., 1., 1.],
          [1., 1., 1., 1.],
          [1., 1., 1., 1.],
          [1., 1., 1., 1.],
          [1., 1., 1., 1.],
          [1., 1., 1., 1.]]], grad_fn=<EmbeddingBackward0>))

**Dropout**

In [18]:
dropout = nn.Dropout(0.1)
dropout

Dropout(p=0.1, inplace=False)

In [19]:
x = dropout(x)
x

tensor([[[1.1111, 1.1111, 1.1111, 0.0000],
         [1.1111, 1.1111, 1.1111, 1.1111],
         [0.0000, 0.0000, 1.1111, 1.1111],
         [1.1111, 1.1111, 1.1111, 1.1111],
         [1.1111, 1.1111, 1.1111, 1.1111],
         [1.1111, 1.1111, 1.1111, 1.1111],
         [1.1111, 1.1111, 1.1111, 1.1111],
         [1.1111, 0.0000, 1.1111, 1.1111]],

        [[1.1111, 1.1111, 1.1111, 1.1111],
         [1.1111, 1.1111, 0.0000, 1.1111],
         [1.1111, 1.1111, 1.1111, 1.1111],
         [1.1111, 1.1111, 1.1111, 1.1111],
         [1.1111, 1.1111, 1.1111, 1.1111],
         [1.1111, 0.0000, 1.1111, 1.1111],
         [1.1111, 1.1111, 1.1111, 1.1111],
         [1.1111, 1.1111, 1.1111, 0.0000]]], grad_fn=<MulBackward0>)

**Recurrent Block**

z is the update gate, r the reset gate, and h~t is the candidate hidden

Short version:

* `x2g` = “input-to-gates” affine: maps the current input $(x_t\in\mathbb{R}^{B\times E})$ to a concatenated vector of length (3H). Splitting it gives $(z_x, r_x, n_x \in \mathbb{R}^{B\times H})$.
* `h2g` = “hidden-to-gates” affine: maps the previous hidden $(h_{t-1}\in\mathbb{R}^{B\times H})$ to another (3H) vector. Splitting it gives $(z_h, r_h, n_h \in \mathbb{R}^{B\times H})$.
* `zx`, `rx`, `nx` are the pre-activation contributions from the input to the GRU’s update, reset, and “new state” (candidate) parts, respectively.
* `zh`, `rh`, `nh` are the corresponding contributions from the previous hidden state.

Putting it together (your code’s math):

$\begin{aligned}
z_x, r_x, n_x &= \mathrm{x2g}(x_t)\ \text{chunked into 3 parts},
z_h, r_h, n_h &= \mathrm{h2g}(h_{t-1})\ \text{chunked into 3 parts},
z &= \sigma(z_x + z_h),
r &= \sigma(r_x + r_h),
\tilde{h}_t &= \tanh!\big(n_x + r \odot n_h\big),
h_t &= (1 - z)\odot \tilde{h}*t + z\odot h*{t-1}.
\end{aligned}$

Notes: (z) is the update gate, (r) the reset gate, and $(\tilde{h}_t)$ is the candidate hidden. Each “chunk” has shape ((B,H)). Also, there’s no `z2g` in that snippet—you probably meant `x2g`; the “2g” just reads “to-gates.”


In [20]:
hidden_size = 5

In [21]:
x2g = nn.Linear(n_embd, 3 * hidden_size, bias=True)
torch.nn.init.constant_(x2g.weight, 0.500)
torch.nn.init.zeros_(x2g.bias)
x2g.weight.size(), x2g.weight, x2g.bias.size(), x2g.bias

(torch.Size([15, 4]),
 Parameter containing:
 tensor([[0.5000, 0.5000, 0.5000, 0.5000],
         [0.5000, 0.5000, 0.5000, 0.5000],
         [0.5000, 0.5000, 0.5000, 0.5000],
         [0.5000, 0.5000, 0.5000, 0.5000],
         [0.5000, 0.5000, 0.5000, 0.5000],
         [0.5000, 0.5000, 0.5000, 0.5000],
         [0.5000, 0.5000, 0.5000, 0.5000],
         [0.5000, 0.5000, 0.5000, 0.5000],
         [0.5000, 0.5000, 0.5000, 0.5000],
         [0.5000, 0.5000, 0.5000, 0.5000],
         [0.5000, 0.5000, 0.5000, 0.5000],
         [0.5000, 0.5000, 0.5000, 0.5000],
         [0.5000, 0.5000, 0.5000, 0.5000],
         [0.5000, 0.5000, 0.5000, 0.5000],
         [0.5000, 0.5000, 0.5000, 0.5000]], requires_grad=True),
 torch.Size([15]),
 Parameter containing:
 tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        requires_grad=True))

In [22]:
h2g = nn.Linear(hidden_size, 3 * hidden_size, bias=True)
torch.nn.init.constant_(h2g.weight, 0.250)
torch.nn.init.zeros_(h2g.bias)
h2g.weight.size(), h2g.weight, h2g.bias.size(), h2g.bias

(torch.Size([15, 5]),
 Parameter containing:
 tensor([[0.2500, 0.2500, 0.2500, 0.2500, 0.2500],
         [0.2500, 0.2500, 0.2500, 0.2500, 0.2500],
         [0.2500, 0.2500, 0.2500, 0.2500, 0.2500],
         [0.2500, 0.2500, 0.2500, 0.2500, 0.2500],
         [0.2500, 0.2500, 0.2500, 0.2500, 0.2500],
         [0.2500, 0.2500, 0.2500, 0.2500, 0.2500],
         [0.2500, 0.2500, 0.2500, 0.2500, 0.2500],
         [0.2500, 0.2500, 0.2500, 0.2500, 0.2500],
         [0.2500, 0.2500, 0.2500, 0.2500, 0.2500],
         [0.2500, 0.2500, 0.2500, 0.2500, 0.2500],
         [0.2500, 0.2500, 0.2500, 0.2500, 0.2500],
         [0.2500, 0.2500, 0.2500, 0.2500, 0.2500],
         [0.2500, 0.2500, 0.2500, 0.2500, 0.2500],
         [0.2500, 0.2500, 0.2500, 0.2500, 0.2500],
         [0.2500, 0.2500, 0.2500, 0.2500, 0.2500]], requires_grad=True),
 torch.Size([15]),
 Parameter containing:
 tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        requires_grad=True))

since first pass, incoming weight (h_t) are seroed

first pass (future will loop

In [23]:
x_0 = x[:, 0, :]
x_0

tensor([[1.1111, 1.1111, 1.1111, 0.0000],
        [1.1111, 1.1111, 1.1111, 1.1111]], grad_fn=<SliceBackward0>)

In [24]:
x_t = x2g(x_0)
zx, rx, nx = x_t.chunk(3, dim=-1)

x_t.size(), 'zx', zx, zx.size(), 'rx',rx, rx.size(),'nx',nx, nx.size()

(torch.Size([2, 15]),
 'zx',
 tensor([[1.6667, 1.6667, 1.6667, 1.6667, 1.6667],
         [2.2222, 2.2222, 2.2222, 2.2222, 2.2222]], grad_fn=<SplitBackward0>),
 torch.Size([2, 5]),
 'rx',
 tensor([[1.6667, 1.6667, 1.6667, 1.6667, 1.6667],
         [2.2222, 2.2222, 2.2222, 2.2222, 2.2222]], grad_fn=<SplitBackward0>),
 torch.Size([2, 5]),
 'nx',
 tensor([[1.6667, 1.6667, 1.6667, 1.6667, 1.6667],
         [2.2222, 2.2222, 2.2222, 2.2222, 2.2222]], grad_fn=<SplitBackward0>),
 torch.Size([2, 5]))

Now we have to do the incoming weights, since this is round 0, they'll be 0 since nothing preceeds the first token. we'll set `h_t` to 0

In [25]:
h_init = torch.zeros(B_batch, hidden_size) 
h_init

tensor([[0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.]])

In [26]:
h_t = h2g(h_init)
zh, rh, nh = h_t.chunk(3, dim=-1)

h_t.size(), 'zh', zh, zh.size(), 'rh',rh, rh.size(),'nh',nh, nh.size()

(torch.Size([2, 15]),
 'zh',
 tensor([[0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.]], grad_fn=<SplitBackward0>),
 torch.Size([2, 5]),
 'rh',
 tensor([[0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.]], grad_fn=<SplitBackward0>),
 torch.Size([2, 5]),
 'nh',
 tensor([[0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.]], grad_fn=<SplitBackward0>),
 torch.Size([2, 5]))

combine the pieces of of the inputs and the previous:

https://docs.pytorch.org/docs/stable/generated/torch.nn.Sigmoid.html

In [27]:
z = torch.sigmoid(zx + zh)
z.size(), z

(torch.Size([2, 5]),
 tensor([[0.8411, 0.8411, 0.8411, 0.8411, 0.8411],
         [0.9022, 0.9022, 0.9022, 0.9022, 0.9022]], grad_fn=<SigmoidBackward0>))

In [28]:
r = torch.sigmoid(rx + rh)
r.size(), r

(torch.Size([2, 5]),
 tensor([[0.8411, 0.8411, 0.8411, 0.8411, 0.8411],
         [0.9022, 0.9022, 0.9022, 0.9022, 0.9022]], grad_fn=<SigmoidBackward0>))

In [29]:
n = torch.tanh(nx + r * nh)

n.size(), n

(torch.Size([2, 5]),
 tensor([[0.9311, 0.9311, 0.9311, 0.9311, 0.9311],
         [0.9768, 0.9768, 0.9768, 0.9768, 0.9768]], grad_fn=<TanhBackward0>))

In [30]:
h_t = (1 - z) * n + z * h_init
h_t.size(), h_t

(torch.Size([2, 5]),
 tensor([[0.1479, 0.1479, 0.1479, 0.1479, 0.1479],
         [0.0955, 0.0955, 0.0955, 0.0955, 0.0955]], grad_fn=<AddBackward0>))

## now do it recursively for the rest of the batch size

first we'll need to keep track of h_t for each loop for training, so let's start collecting them. 

In [31]:
hs = []
hs.append(h_t.unsqueeze(1))
hs

[tensor([[[0.1479, 0.1479, 0.1479, 0.1479, 0.1479]],
 
         [[0.0955, 0.0955, 0.0955, 0.0955, 0.0955]]],
        grad_fn=<UnsqueezeBackward0>)]

start at 1 since we did the "0" pass

In [32]:
for t in range(1,T):
    x_t = x[:, t, :]
    h_prev = h_t
    zx, rx, nx = x2g(x_t).chunk(3, dim=-1)
    zh, rh, nh = h2g(h_prev).chunk(3, dim=-1)
    z = torch.sigmoid(zx + zh)
    r = torch.sigmoid(rx + rh)
    n = torch.tanh(nx + r * nh)
    h_t = (1 - z) * n + z * h_prev
    
    print(f't: {t}')
    print(h_t)
    hs.append(h_t.unsqueeze(1))

t: 1
tensor([[0.2170, 0.2170, 0.2170, 0.2170, 0.2170],
        [0.2172, 0.2172, 0.2172, 0.2172, 0.2172]], grad_fn=<AddBackward0>)
t: 2
tensor([[0.3477, 0.3477, 0.3477, 0.3477, 0.3477],
        [0.2759, 0.2759, 0.2759, 0.2759, 0.2759]], grad_fn=<AddBackward0>)
t: 3
tensor([[0.3898, 0.3898, 0.3898, 0.3898, 0.3898],
        [0.3266, 0.3266, 0.3266, 0.3266, 0.3266]], grad_fn=<AddBackward0>)
t: 4
tensor([[0.4273, 0.4273, 0.4273, 0.4273, 0.4273],
        [0.3711, 0.3711, 0.3711, 0.3711, 0.3711]], grad_fn=<AddBackward0>)
t: 5
tensor([[0.4610, 0.4610, 0.4610, 0.4610, 0.4610],
        [0.4346, 0.4346, 0.4346, 0.4346, 0.4346]], grad_fn=<AddBackward0>)
t: 6
tensor([[0.4915, 0.4915, 0.4915, 0.4915, 0.4915],
        [0.4676, 0.4676, 0.4676, 0.4676, 0.4676]], grad_fn=<AddBackward0>)
t: 7
tensor([[0.5365, 0.5365, 0.5365, 0.5365, 0.5365],
        [0.5160, 0.5160, 0.5160, 0.5160, 0.5160]], grad_fn=<AddBackward0>)


In [33]:
hs

[tensor([[[0.1479, 0.1479, 0.1479, 0.1479, 0.1479]],
 
         [[0.0955, 0.0955, 0.0955, 0.0955, 0.0955]]],
        grad_fn=<UnsqueezeBackward0>),
 tensor([[[0.2170, 0.2170, 0.2170, 0.2170, 0.2170]],
 
         [[0.2172, 0.2172, 0.2172, 0.2172, 0.2172]]],
        grad_fn=<UnsqueezeBackward0>),
 tensor([[[0.3477, 0.3477, 0.3477, 0.3477, 0.3477]],
 
         [[0.2759, 0.2759, 0.2759, 0.2759, 0.2759]]],
        grad_fn=<UnsqueezeBackward0>),
 tensor([[[0.3898, 0.3898, 0.3898, 0.3898, 0.3898]],
 
         [[0.3266, 0.3266, 0.3266, 0.3266, 0.3266]]],
        grad_fn=<UnsqueezeBackward0>),
 tensor([[[0.4273, 0.4273, 0.4273, 0.4273, 0.4273]],
 
         [[0.3711, 0.3711, 0.3711, 0.3711, 0.3711]]],
        grad_fn=<UnsqueezeBackward0>),
 tensor([[[0.4610, 0.4610, 0.4610, 0.4610, 0.4610]],
 
         [[0.4346, 0.4346, 0.4346, 0.4346, 0.4346]]],
        grad_fn=<UnsqueezeBackward0>),
 tensor([[[0.4915, 0.4915, 0.4915, 0.4915, 0.4915]],
 
         [[0.4676, 0.4676, 0.4676, 0.4676, 0.4676]]],
   

Combine out recurring inputs into the 2 different batches

In [34]:
x = torch.cat(hs, dim=1) 
x

tensor([[[0.1479, 0.1479, 0.1479, 0.1479, 0.1479],
         [0.2170, 0.2170, 0.2170, 0.2170, 0.2170],
         [0.3477, 0.3477, 0.3477, 0.3477, 0.3477],
         [0.3898, 0.3898, 0.3898, 0.3898, 0.3898],
         [0.4273, 0.4273, 0.4273, 0.4273, 0.4273],
         [0.4610, 0.4610, 0.4610, 0.4610, 0.4610],
         [0.4915, 0.4915, 0.4915, 0.4915, 0.4915],
         [0.5365, 0.5365, 0.5365, 0.5365, 0.5365]],

        [[0.0955, 0.0955, 0.0955, 0.0955, 0.0955],
         [0.2172, 0.2172, 0.2172, 0.2172, 0.2172],
         [0.2759, 0.2759, 0.2759, 0.2759, 0.2759],
         [0.3266, 0.3266, 0.3266, 0.3266, 0.3266],
         [0.3711, 0.3711, 0.3711, 0.3711, 0.3711],
         [0.4346, 0.4346, 0.4346, 0.4346, 0.4346],
         [0.4676, 0.4676, 0.4676, 0.4676, 0.4676],
         [0.5160, 0.5160, 0.5160, 0.5160, 0.5160]]], grad_fn=<CatBackward0>)

**Dropout** to fight vanishing / exploding gradient

In [35]:
x = dropout(x)
x.size(), x

(torch.Size([2, 8, 5]),
 tensor([[[0.1644, 0.1644, 0.1644, 0.1644, 0.1644],
          [0.2411, 0.2411, 0.2411, 0.2411, 0.2411],
          [0.3864, 0.3864, 0.3864, 0.3864, 0.3864],
          [0.4331, 0.4331, 0.4331, 0.4331, 0.4331],
          [0.4748, 0.0000, 0.4748, 0.4748, 0.4748],
          [0.5122, 0.5122, 0.5122, 0.0000, 0.5122],
          [0.5461, 0.5461, 0.5461, 0.5461, 0.5461],
          [0.5961, 0.5961, 0.0000, 0.5961, 0.5961]],
 
         [[0.1061, 0.1061, 0.1061, 0.1061, 0.1061],
          [0.2414, 0.2414, 0.2414, 0.2414, 0.2414],
          [0.0000, 0.3065, 0.3065, 0.3065, 0.3065],
          [0.3629, 0.3629, 0.3629, 0.3629, 0.3629],
          [0.4124, 0.4124, 0.4124, 0.4124, 0.4124],
          [0.4829, 0.4829, 0.4829, 0.4829, 0.4829],
          [0.5196, 0.0000, 0.5196, 0.5196, 0.5196],
          [0.5733, 0.5733, 0.0000, 0.0000, 0.5733]]], grad_fn=<MulBackward0>))

**Output Head**
projects down from the hiden size to the vocab for us to get logits. 

In [36]:
lm_head = nn.Linear(hidden_size, vocab_size, bias=False)
torch.nn.init.ones_(lm_head.weight)
lm_head.weight.size(), lm_head.weight

(torch.Size([36, 5]),
 Parameter containing:
 tensor([[1., 1., 1., 1., 1.],
         [1., 1., 1., 1., 1.],
         [1., 1., 1., 1., 1.],
         [1., 1., 1., 1., 1.],
         [1., 1., 1., 1., 1.],
         [1., 1., 1., 1., 1.],
         [1., 1., 1., 1., 1.],
         [1., 1., 1., 1., 1.],
         [1., 1., 1., 1., 1.],
         [1., 1., 1., 1., 1.],
         [1., 1., 1., 1., 1.],
         [1., 1., 1., 1., 1.],
         [1., 1., 1., 1., 1.],
         [1., 1., 1., 1., 1.],
         [1., 1., 1., 1., 1.],
         [1., 1., 1., 1., 1.],
         [1., 1., 1., 1., 1.],
         [1., 1., 1., 1., 1.],
         [1., 1., 1., 1., 1.],
         [1., 1., 1., 1., 1.],
         [1., 1., 1., 1., 1.],
         [1., 1., 1., 1., 1.],
         [1., 1., 1., 1., 1.],
         [1., 1., 1., 1., 1.],
         [1., 1., 1., 1., 1.],
         [1., 1., 1., 1., 1.],
         [1., 1., 1., 1., 1.],
         [1., 1., 1., 1., 1.],
         [1., 1., 1., 1., 1.],
         [1., 1., 1., 1., 1.],
         [1., 1., 1., 1.,

In [37]:
logits = lm_head(x)

logits.shape, logits

(torch.Size([2, 8, 36]),
 tensor([[[0.8218, 0.8218, 0.8218, 0.8218, 0.8218, 0.8218, 0.8218, 0.8218,
           0.8218, 0.8218, 0.8218, 0.8218, 0.8218, 0.8218, 0.8218, 0.8218,
           0.8218, 0.8218, 0.8218, 0.8218, 0.8218, 0.8218, 0.8218, 0.8218,
           0.8218, 0.8218, 0.8218, 0.8218, 0.8218, 0.8218, 0.8218, 0.8218,
           0.8218, 0.8218, 0.8218, 0.8218],
          [1.2053, 1.2053, 1.2053, 1.2053, 1.2053, 1.2053, 1.2053, 1.2053,
           1.2053, 1.2053, 1.2053, 1.2053, 1.2053, 1.2053, 1.2053, 1.2053,
           1.2053, 1.2053, 1.2053, 1.2053, 1.2053, 1.2053, 1.2053, 1.2053,
           1.2053, 1.2053, 1.2053, 1.2053, 1.2053, 1.2053, 1.2053, 1.2053,
           1.2053, 1.2053, 1.2053, 1.2053],
          [1.9318, 1.9318, 1.9318, 1.9318, 1.9318, 1.9318, 1.9318, 1.9318,
           1.9318, 1.9318, 1.9318, 1.9318, 1.9318, 1.9318, 1.9318, 1.9318,
           1.9318, 1.9318, 1.9318, 1.9318, 1.9318, 1.9318, 1.9318, 1.9318,
           1.9318, 1.9318, 1.9318, 1.9318, 1.9318, 1.9318, 1.9

**Loss**

In [38]:
import torch.nn.functional as F

In [39]:
y_flat = y.view(-1)
y_flat.shape, y_flat

(torch.Size([16]),
 tensor([15, 32,  9,  5, 20, 30, 15, 11,  9,  6, 20,  5,  0, 13, 21,  0]))

In [40]:
logits_flat = logits.view(-1, logits.size(-1))
logits_flat.shape, logits_flat

(torch.Size([16, 36]),
 tensor([[0.8218, 0.8218, 0.8218, 0.8218, 0.8218, 0.8218, 0.8218, 0.8218, 0.8218,
          0.8218, 0.8218, 0.8218, 0.8218, 0.8218, 0.8218, 0.8218, 0.8218, 0.8218,
          0.8218, 0.8218, 0.8218, 0.8218, 0.8218, 0.8218, 0.8218, 0.8218, 0.8218,
          0.8218, 0.8218, 0.8218, 0.8218, 0.8218, 0.8218, 0.8218, 0.8218, 0.8218],
         [1.2053, 1.2053, 1.2053, 1.2053, 1.2053, 1.2053, 1.2053, 1.2053, 1.2053,
          1.2053, 1.2053, 1.2053, 1.2053, 1.2053, 1.2053, 1.2053, 1.2053, 1.2053,
          1.2053, 1.2053, 1.2053, 1.2053, 1.2053, 1.2053, 1.2053, 1.2053, 1.2053,
          1.2053, 1.2053, 1.2053, 1.2053, 1.2053, 1.2053, 1.2053, 1.2053, 1.2053],
         [1.9318, 1.9318, 1.9318, 1.9318, 1.9318, 1.9318, 1.9318, 1.9318, 1.9318,
          1.9318, 1.9318, 1.9318, 1.9318, 1.9318, 1.9318, 1.9318, 1.9318, 1.9318,
          1.9318, 1.9318, 1.9318, 1.9318, 1.9318, 1.9318, 1.9318, 1.9318, 1.9318,
          1.9318, 1.9318, 1.9318, 1.9318, 1.9318, 1.9318, 1.9318, 1.9318,

In [41]:
loss = F.cross_entropy(logits_flat, y_flat)
loss.shape, loss

(torch.Size([]), tensor(3.5835, grad_fn=<NllLossBackward0>))

## Back Propogation

In [42]:
lm_head.zero_grad()
h2g.zero_grad()
x2g.zero_grad()
wte.zero_grad()


# validate gradients
lm_head.weight.grad, wte.weight.grad

(None, None)

In [43]:
loss.backward()

In [44]:
lm_head.weight.grad, wte.weight.grad

(tensor([[-0.0511, -0.0523, -0.0168, -0.0166, -0.0506],
         [ 0.0105,  0.0093,  0.0090,  0.0092,  0.0110],
         [ 0.0105,  0.0093,  0.0090,  0.0092,  0.0110],
         [ 0.0105,  0.0093,  0.0090,  0.0092,  0.0110],
         [ 0.0105,  0.0093,  0.0090,  0.0092,  0.0110],
         [-0.0392, -0.0404, -0.0407, -0.0406, -0.0387],
         [-0.0046, -0.0058, -0.0061, -0.0059, -0.0040],
         [ 0.0105,  0.0093,  0.0090,  0.0092,  0.0110],
         [ 0.0105,  0.0093,  0.0090,  0.0092,  0.0110],
         [-0.0203, -0.0215, -0.0218, -0.0216, -0.0197],
         [ 0.0105,  0.0093,  0.0090,  0.0092,  0.0110],
         [-0.0267, -0.0279,  0.0090, -0.0281, -0.0262],
         [ 0.0105,  0.0093,  0.0090,  0.0092,  0.0110],
         [-0.0197, -0.0209, -0.0212, -0.0210, -0.0191],
         [ 0.0105,  0.0093,  0.0090,  0.0092,  0.0110],
         [-0.0339, -0.0351, -0.0354, -0.0352, -0.0334],
         [ 0.0105,  0.0093,  0.0090,  0.0092,  0.0110],
         [ 0.0105,  0.0093,  0.0090,  0.0092,  0

## Learning 

In [45]:
## Huge learning rate to emphasize
learning_rate = 5.000

In [46]:
with torch.no_grad():
    lm_head.weight -= learning_rate * lm_head.weight.grad
lm_head.weight

Parameter containing:
tensor([[1.2555, 1.2615, 1.0838, 1.0831, 1.2528],
        [0.9475, 0.9534, 0.9549, 0.9542, 0.9448],
        [0.9475, 0.9534, 0.9549, 0.9542, 0.9448],
        [0.9475, 0.9534, 0.9549, 0.9542, 0.9448],
        [0.9475, 0.9534, 0.9549, 0.9542, 0.9448],
        [1.1962, 1.2022, 1.2037, 1.2030, 1.1936],
        [1.0229, 1.0289, 1.0304, 1.0297, 1.0202],
        [0.9475, 0.9534, 0.9549, 0.9542, 0.9448],
        [0.9475, 0.9534, 0.9549, 0.9542, 0.9448],
        [1.1014, 1.1073, 1.1088, 1.1081, 1.0987],
        [0.9475, 0.9534, 0.9549, 0.9542, 0.9448],
        [1.1337, 1.1397, 0.9549, 1.1405, 1.1311],
        [0.9475, 0.9534, 0.9549, 0.9542, 0.9448],
        [1.0984, 1.1043, 1.1059, 1.1051, 1.0957],
        [0.9475, 0.9534, 0.9549, 0.9542, 0.9448],
        [1.1695, 1.1755, 1.1770, 1.1762, 1.1668],
        [0.9475, 0.9534, 0.9549, 0.9542, 0.9448],
        [0.9475, 0.9534, 0.9549, 0.9542, 0.9448],
        [0.9475, 0.9534, 0.9549, 0.9542, 0.9448],
        [0.9475, 0.9534, 0.9

In [47]:
with torch.no_grad():
    h2g.weight -= learning_rate * h2g.weight.grad
    h2g.bias -= learning_rate * h2g.bias.grad
h2g.weight, h2g.bias

(Parameter containing:
 tensor([[0.2500, 0.2500, 0.2500, 0.2500, 0.2500],
         [0.2500, 0.2500, 0.2500, 0.2500, 0.2500],
         [0.2500, 0.2500, 0.2500, 0.2500, 0.2500],
         [0.2500, 0.2500, 0.2500, 0.2500, 0.2500],
         [0.2500, 0.2500, 0.2500, 0.2500, 0.2500],
         [0.2500, 0.2500, 0.2500, 0.2500, 0.2500],
         [0.2500, 0.2500, 0.2500, 0.2500, 0.2500],
         [0.2500, 0.2500, 0.2500, 0.2500, 0.2500],
         [0.2500, 0.2500, 0.2500, 0.2500, 0.2500],
         [0.2500, 0.2500, 0.2500, 0.2500, 0.2500],
         [0.2500, 0.2500, 0.2500, 0.2500, 0.2500],
         [0.2500, 0.2500, 0.2500, 0.2500, 0.2500],
         [0.2500, 0.2500, 0.2500, 0.2500, 0.2500],
         [0.2500, 0.2500, 0.2500, 0.2500, 0.2500],
         [0.2500, 0.2500, 0.2500, 0.2500, 0.2500]], requires_grad=True),
 Parameter containing:
 tensor([ 2.4692e-07,  2.2701e-07,  2.0391e-07,  1.9728e-07,  2.5823e-07,
         -9.2556e-10, -8.0872e-10, -7.6041e-10, -7.1028e-10, -9.3874e-10,
         -2.8969e-0

In [48]:
with torch.no_grad():
    x2g.weight -= learning_rate * x2g.weight.grad
    x2g.bias -= learning_rate * x2g.bias.grad
x2g.weight, x2g.bias

(Parameter containing:
 tensor([[0.5000, 0.5000, 0.5000, 0.5000],
         [0.5000, 0.5000, 0.5000, 0.5000],
         [0.5000, 0.5000, 0.5000, 0.5000],
         [0.5000, 0.5000, 0.5000, 0.5000],
         [0.5000, 0.5000, 0.5000, 0.5000],
         [0.5000, 0.5000, 0.5000, 0.5000],
         [0.5000, 0.5000, 0.5000, 0.5000],
         [0.5000, 0.5000, 0.5000, 0.5000],
         [0.5000, 0.5000, 0.5000, 0.5000],
         [0.5000, 0.5000, 0.5000, 0.5000],
         [0.5000, 0.5000, 0.5000, 0.5000],
         [0.5000, 0.5000, 0.5000, 0.5000],
         [0.5000, 0.5000, 0.5000, 0.5000],
         [0.5000, 0.5000, 0.5000, 0.5000],
         [0.5000, 0.5000, 0.5000, 0.5000]], requires_grad=True),
 Parameter containing:
 tensor([ 2.4692e-07,  2.2701e-07,  2.0391e-07,  1.9728e-07,  2.5823e-07,
         -9.2556e-10, -8.0872e-10, -7.6041e-10, -7.1028e-10, -9.3874e-10,
         -3.4525e-08, -3.1154e-08, -2.9693e-08, -2.8006e-08, -3.5573e-08],
        requires_grad=True))

In [49]:
with torch.no_grad():
    wte.weight -= learning_rate * wte.weight.grad
wte.weight

Parameter containing:
tensor([[1.0000, 1.0000, 1.0000, 1.0000],
        [1.0000, 1.0000, 1.0000, 1.0000],
        [1.0000, 1.0000, 1.0000, 1.0000],
        [1.0000, 1.0000, 1.0000, 1.0000],
        [1.0000, 1.0000, 1.0000, 1.0000],
        [1.0000, 1.0000, 1.0000, 1.0000],
        [1.0000, 1.0000, 1.0000, 1.0000],
        [1.0000, 1.0000, 1.0000, 1.0000],
        [1.0000, 1.0000, 1.0000, 1.0000],
        [1.0000, 1.0000, 1.0000, 1.0000],
        [1.0000, 1.0000, 1.0000, 1.0000],
        [1.0000, 1.0000, 1.0000, 1.0000],
        [1.0000, 1.0000, 1.0000, 1.0000],
        [1.0000, 1.0000, 1.0000, 1.0000],
        [1.0000, 1.0000, 1.0000, 1.0000],
        [1.0000, 1.0000, 1.0000, 1.0000],
        [1.0000, 1.0000, 1.0000, 1.0000],
        [1.0000, 1.0000, 1.0000, 1.0000],
        [1.0000, 1.0000, 1.0000, 1.0000],
        [1.0000, 1.0000, 1.0000, 1.0000],
        [1.0000, 1.0000, 1.0000, 1.0000],
        [1.0000, 1.0000, 1.0000, 1.0000],
        [1.0000, 1.0000, 1.0000, 1.0000],
        [1.0

## Forward Pass with Updated Weights

In [50]:
x_2 = tok_for_training[:-1].view(B, T)
x_2, y

(tensor([[35, 15, 32,  9,  5, 20, 30, 15],
         [11,  9,  6, 20,  5,  0, 13, 21]]),
 tensor([[15, 32,  9,  5, 20, 30, 15, 11],
         [ 9,  6, 20,  5,  0, 13, 21,  0]]))

## Input projection

In [51]:
x = wte(x_2)
x.shape, x

(torch.Size([2, 8, 4]),
 tensor([[[1.0000, 1.0000, 1.0000, 1.0000],
          [1.0000, 1.0000, 1.0000, 1.0000],
          [1.0000, 1.0000, 1.0000, 1.0000],
          [1.0000, 1.0000, 1.0000, 1.0000],
          [1.0000, 1.0000, 1.0000, 1.0000],
          [1.0000, 1.0000, 1.0000, 1.0000],
          [1.0000, 1.0000, 1.0000, 1.0000],
          [1.0000, 1.0000, 1.0000, 1.0000]],
 
         [[1.0000, 1.0000, 1.0000, 1.0000],
          [1.0000, 1.0000, 1.0000, 1.0000],
          [1.0000, 1.0000, 1.0000, 1.0000],
          [1.0000, 1.0000, 1.0000, 1.0000],
          [1.0000, 1.0000, 1.0000, 1.0000],
          [1.0000, 1.0000, 1.0000, 1.0000],
          [1.0000, 1.0000, 1.0000, 1.0000],
          [1.0000, 1.0000, 1.0000, 1.0000]]], grad_fn=<EmbeddingBackward0>))

**Dropout**

In [52]:
x = dropout(x)
x


tensor([[[1.1111, 1.1111, 1.1111, 1.1111],
         [1.1111, 1.1111, 1.1111, 1.1111],
         [1.1111, 1.1111, 1.1111, 0.0000],
         [1.1111, 1.1111, 1.1111, 1.1111],
         [1.1111, 1.1111, 1.1111, 1.1111],
         [1.1111, 1.1111, 1.1111, 1.1111],
         [1.1111, 1.1111, 1.1111, 1.1111],
         [1.1111, 1.1111, 1.1111, 1.1111]],

        [[1.1111, 1.1111, 1.1111, 1.1111],
         [1.1111, 1.1111, 1.1111, 1.1111],
         [1.1111, 1.1111, 1.1111, 1.1111],
         [1.1111, 1.1111, 1.1111, 1.1111],
         [1.1111, 1.1111, 1.1111, 1.1111],
         [1.1111, 1.1111, 1.1111, 0.0000],
         [0.0000, 1.1111, 0.0000, 1.1111],
         [1.1111, 1.1111, 1.1111, 1.1111]]], grad_fn=<MulBackward0>)

**Recurrent Block** Collapsed Together

h_t still resets to 0.

In [53]:
h_t = torch.zeros(B_batch, hidden_size) 
hs = []
h_t

tensor([[0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.]])

In [54]:
for t in range(T):
    x_t = x[:, t, :]
    h_prev = h_t
    zx, rx, nx = x2g(x_t).chunk(3, dim=-1)
    zh, rh, nh = h2g(h_prev).chunk(3, dim=-1)
    z = torch.sigmoid(zx + zh)
    r = torch.sigmoid(rx + rh)
    n = torch.tanh(nx + r * nh)
    h_t = (1 - z) * n + z * h_prev
    
    print(f't: {t}')
    print(h_t)
    hs.append(h_t.unsqueeze(1))

hs

t: 0
tensor([[0.0955, 0.0955, 0.0955, 0.0955, 0.0955],
        [0.0955, 0.0955, 0.0955, 0.0955, 0.0955]], grad_fn=<AddBackward0>)
t: 1
tensor([[0.1732, 0.1732, 0.1732, 0.1732, 0.1732],
        [0.1732, 0.1732, 0.1732, 0.1732, 0.1732]], grad_fn=<AddBackward0>)
t: 2
tensor([[0.2761, 0.2761, 0.2761, 0.2761, 0.2761],
        [0.2383, 0.2383, 0.2383, 0.2383, 0.2383]], grad_fn=<AddBackward0>)
t: 3
tensor([[0.3268, 0.3268, 0.3268, 0.3268, 0.3268],
        [0.2940, 0.2940, 0.2940, 0.2940, 0.2940]], grad_fn=<AddBackward0>)
t: 4
tensor([[0.3713, 0.3713, 0.3713, 0.3713, 0.3713],
        [0.3425, 0.3425, 0.3425, 0.3425, 0.3425]], grad_fn=<AddBackward0>)
t: 5
tensor([[0.4108, 0.4108, 0.4108, 0.4108, 0.4108],
        [0.4110, 0.4110, 0.4110, 0.4110, 0.4110]], grad_fn=<AddBackward0>)
t: 6
tensor([[0.4461, 0.4461, 0.4461, 0.4461, 0.4461],
        [0.4934, 0.4934, 0.4934, 0.4934, 0.4934]], grad_fn=<AddBackward0>)
t: 7
tensor([[0.4780, 0.4780, 0.4780, 0.4780, 0.4780],
        [0.5210, 0.5210, 0.5210, 0.

[tensor([[[0.0955, 0.0955, 0.0955, 0.0955, 0.0955]],
 
         [[0.0955, 0.0955, 0.0955, 0.0955, 0.0955]]],
        grad_fn=<UnsqueezeBackward0>),
 tensor([[[0.1732, 0.1732, 0.1732, 0.1732, 0.1732]],
 
         [[0.1732, 0.1732, 0.1732, 0.1732, 0.1732]]],
        grad_fn=<UnsqueezeBackward0>),
 tensor([[[0.2761, 0.2761, 0.2761, 0.2761, 0.2761]],
 
         [[0.2383, 0.2383, 0.2383, 0.2383, 0.2383]]],
        grad_fn=<UnsqueezeBackward0>),
 tensor([[[0.3268, 0.3268, 0.3268, 0.3268, 0.3268]],
 
         [[0.2940, 0.2940, 0.2940, 0.2940, 0.2940]]],
        grad_fn=<UnsqueezeBackward0>),
 tensor([[[0.3713, 0.3713, 0.3713, 0.3713, 0.3713]],
 
         [[0.3425, 0.3425, 0.3425, 0.3425, 0.3425]]],
        grad_fn=<UnsqueezeBackward0>),
 tensor([[[0.4108, 0.4108, 0.4108, 0.4108, 0.4108]],
 
         [[0.4110, 0.4110, 0.4110, 0.4110, 0.4110]]],
        grad_fn=<UnsqueezeBackward0>),
 tensor([[[0.4461, 0.4461, 0.4461, 0.4461, 0.4461]],
 
         [[0.4934, 0.4934, 0.4934, 0.4934, 0.4934]]],
   

combine weights back together

In [55]:
x = torch.cat(hs, dim=1) 
x.size(), x

(torch.Size([2, 8, 5]),
 tensor([[[0.0955, 0.0955, 0.0955, 0.0955, 0.0955],
          [0.1732, 0.1732, 0.1732, 0.1732, 0.1732],
          [0.2761, 0.2761, 0.2761, 0.2761, 0.2761],
          [0.3268, 0.3268, 0.3268, 0.3268, 0.3268],
          [0.3713, 0.3713, 0.3713, 0.3713, 0.3713],
          [0.4108, 0.4108, 0.4108, 0.4108, 0.4108],
          [0.4461, 0.4461, 0.4461, 0.4461, 0.4461],
          [0.4780, 0.4780, 0.4780, 0.4780, 0.4780]],
 
         [[0.0955, 0.0955, 0.0955, 0.0955, 0.0955],
          [0.1732, 0.1732, 0.1732, 0.1732, 0.1732],
          [0.2383, 0.2383, 0.2383, 0.2383, 0.2383],
          [0.2940, 0.2940, 0.2940, 0.2940, 0.2940],
          [0.3425, 0.3425, 0.3425, 0.3425, 0.3425],
          [0.4110, 0.4110, 0.4110, 0.4110, 0.4110],
          [0.4934, 0.4934, 0.4934, 0.4934, 0.4934],
          [0.5210, 0.5210, 0.5210, 0.5210, 0.5210]]], grad_fn=<CatBackward0>))

**Dropout**

In [56]:
x = dropout(x)
x


tensor([[[0.1061, 0.1061, 0.1061, 0.1061, 0.1061],
         [0.1925, 0.1925, 0.1925, 0.1925, 0.1925],
         [0.3067, 0.3067, 0.3067, 0.3067, 0.3067],
         [0.3631, 0.3631, 0.3631, 0.3631, 0.3631],
         [0.4125, 0.4125, 0.4125, 0.4125, 0.4125],
         [0.4564, 0.4564, 0.4564, 0.4564, 0.4564],
         [0.4957, 0.4957, 0.4957, 0.4957, 0.4957],
         [0.5311, 0.5311, 0.5311, 0.5311, 0.5311]],

        [[0.1061, 0.1061, 0.1061, 0.1061, 0.1061],
         [0.0000, 0.1925, 0.1925, 0.1925, 0.1925],
         [0.2648, 0.2648, 0.2648, 0.2648, 0.2648],
         [0.3267, 0.0000, 0.3267, 0.3267, 0.0000],
         [0.3805, 0.3805, 0.0000, 0.3805, 0.3805],
         [0.4566, 0.0000, 0.4566, 0.4566, 0.4566],
         [0.5483, 0.5483, 0.5483, 0.5483, 0.5483],
         [0.5789, 0.5789, 0.5789, 0.5789, 0.5789]]], grad_fn=<MulBackward0>)

**Head**

In [57]:
logits = lm_head(x)
logits.shape, logits

(torch.Size([2, 8, 36]),
 tensor([[[0.6300, 0.5046, 0.5046, 0.5046, 0.5046, 0.6365, 0.5446, 0.5046,
           0.5046, 0.5862, 0.5046, 0.5836, 0.5046, 0.5846, 0.5046, 0.6224,
           0.5046, 0.5046, 0.5046, 0.5046, 0.6082, 0.5735, 0.5046, 0.5046,
           0.5046, 0.5046, 0.5046, 0.5046, 0.5046, 0.5046, 0.5725, 0.5046,
           0.5445, 0.5046, 0.5046, 0.5046],
          [1.1426, 0.9151, 0.9151, 0.9151, 0.9151, 1.1545, 0.9877, 0.9151,
           0.9151, 1.0632, 0.9151, 1.0585, 0.9151, 1.0604, 0.9151, 1.1288,
           0.9151, 0.9151, 0.9151, 0.9151, 1.1031, 1.0401, 0.9151, 0.9151,
           0.9151, 0.9151, 0.9151, 0.9151, 0.9151, 0.9151, 1.0384, 0.9151,
           0.9876, 0.9151, 0.9151, 0.9151],
          [1.8210, 1.4585, 1.4585, 1.4585, 1.4585, 1.8400, 1.5741, 1.4585,
           1.4585, 1.6945, 1.4585, 1.6870, 1.4585, 1.6899, 1.4585, 1.7990,
           1.4585, 1.4585, 1.4585, 1.4585, 1.7580, 1.6577, 1.4585, 1.4585,
           1.4585, 1.4585, 1.4585, 1.4585, 1.4585, 1.4585, 1.6

### Updated Loss calculation

Now we'll calculate the updated loss.  Our first pass's loss was 3.5835. Since we're passing through the same example and used a fairly high learning rate we should see a significant improvement with just 1 learning pass. 

In [58]:
loss

tensor(3.5835, grad_fn=<NllLossBackward0>)

In [59]:
y_flat = y.view(-1)
logits_flat = logits.view(-1, logits.size(-1))
updated_loss = F.cross_entropy(logits_flat, y_flat)
print(updated_loss.shape, updated_loss)

torch.Size([]) tensor(3.3731, grad_fn=<NllLossBackward0>)


In [60]:
f'1 round of training resulted in an loss improvment of {loss.item() - updated_loss.item():.4f}'

'1 round of training resulted in an loss improvment of 0.2105'

# SUCCESS!
Our training improved the loss by about **~6%** (amount may vary since we didn't set a seed). There are flaws with this, mainly passing the same example through a second time, but this helps show the fundamentals of what learning does inside a GPT-2 style model. 